# 패키지 불러오기

In [ ]:
import tensorflow as tf
import model as ml
import data
import numpy as np
import os
import sys

from configs import DEFINES

# 사전 만들기

In [ ]:
DATA_OUT_PATH = './data_out/'
data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH)
os.makedirs(data_out_path, exist_ok=True)
char2idx,  idx2char, vocabulary_length = data.load_vocabulary()

# 데이터 만들기

In [ ]:
train_input, train_label, eval_input, eval_label = data.load_data()

#TODO3: 실행 안됨, 실행 후 지속적으로 리뷰하겠음

# 훈련셋 인코딩 만드는 부분이다.
train_input_enc, train_input_enc_length = data.enc_processing(train_input, char2idx)
# 훈련셋 디코딩 입력 부분 만드는 부분이다.
train_output_dec, train_output_dec_length = data.dec_input_processing(train_label, char2idx) #TODO1 실행 안되어 확인 필요(AttributeError: module 'data' has no attribute 'dec_output_processing)
# 훈련셋 디코딩 출력 부분 만드는 부분이다.
train_target_dec = data.dec_target_processing(train_label, char2idx)

# 평가셋 인코딩 만드는 부분이다.
eval_input_enc, eval_input_enc_length = data.enc_processing(eval_input,char2idx)
# 평가셋 인코딩 만드는 부분이다.
eval_output_dec, eval_output_dec_length = data.dec_input_processing(eval_label, char2idx)
# 평가셋 인코딩 만드는 부분이다.
eval_target_dec = data.dec_target_processing(eval_label, char2idx)

# 체크 포인트 경로 만들기

In [ ]:
# 현재 경로'./'에 현재 경로 하부에 
# 체크 포인트를 저장한 디렉토리를 설정한다.
check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)
# 디렉토리를 만드는 함수이며 두번째 인자 exist_ok가 
# True이면 디렉토리가 이미 존재해도 OSError가 
# 발생하지 않는다.
# exist_ok가 False이면 이미 존재하면 
# OSError가 발생한다.
os.makedirs(check_point_path, exist_ok=True)

# 에스티 메이터 구성

In [ ]:
# 에스티메이터 구성한다.

# TODO2: 왜 분류모델이라고 적혀있나요???? s2s이 아닌가요?

classifier = tf.estimator.Estimator(
        model_fn=ml.model, # 모델 등록한다.
        model_dir=DEFINES.check_point_path, # 체크포인트 위치 등록한다.
        params={ # 모델 쪽으로 파라메터 전달한다.
            'hidden_size': DEFINES.hidden_size, # 가중치 크기 설정한다.
            'layer_size': DEFINES.layer_size, # 멀티 레이어 층 개수를 설정한다.
            'learning_rate': DEFINES.learning_rate, # 학습율 설정한다. 
            'vocabulary_length': vocabulary_length, # 딕셔너리 크기를 설정한다.
            'embedding_size': DEFINES.embedding_size, # 임베딩 크기를 설정한다.
            'embedding': DEFINES.embedding, # 임베딩 사용 유무를 설정한다.
            'multilayer': DEFINES.multilayer, # 멀티 레이어 사용 유무를 설정한다.
        })

# 학습 실행

In [ ]:
# 학습 실행
classifier.train(input_fn=lambda:data.train_input_fn(
    train_input_enc, train_output_dec, train_target_dec,  DEFINES.batch_size), steps=DEFINES.train_steps)

# 평가 하기

In [ ]:
eval_result = classifier.evaluate(input_fn=lambda:data.eval_input_fn(
    eval_input_enc, eval_output_dec, eval_target_dec,  DEFINES.batch_size))
print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

# 테스트 하기

In [ ]:
# 테스트용 데이터 만드는 부분이다.
# 인코딩 부분 만든다.
predic_input_enc, predic_input_enc_length = data.enc_processing(["가끔 궁금해"], char2idx)
# 학습 과정이 아니므로 디코딩 입력은 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
predic_output_dec, predic_output_decLength = data.dec_input_processing([""], char2idx)       
# 학습 과정이 아니므로 디코딩 출력 부분도 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
predic_target_dec = data.dec_target_processing([""], char2idx)      

# 예측을 하는 부분이다.
predictions = classifier.predict(
    input_fn=lambda:data.eval_input_fn(predic_input_enc, predic_output_dec, predic_target_dec, DEFINES.batch_size))

# 예측한 값을 인지 할 수 있도록 
# 텍스트로 변경하는 부분이다.
data.pred2string(predictions, idx2char)